## 2. mongodb 인덱스(index)
* 검색을 더 빠르게 수행하고자 만든 추가적인 data structure
    * index 데이터 구조가 없다면, 컬렉션에 있는 데이터를 하나씩 조회하는 방식으로 검색이 됨
    
### 2.1 기본인덱스 _id
* 모든 mongodb 컬렉션은 기본적으로 _id 필드를 기반으로 기본 인덱스가 생성됨

#### mongodb 접속 기본 코드

In [1]:
from bs4 import BeautifulSoup
import requests
import pymongo
import re                   

connection = pymongo.MongoClient()
actor_db = connection.cine21
actor_collection = actor_db.actor_collection

actor_collection.find_one({})
docs = actor_collection.find({}).limit(3)

for doc in docs:
    print(doc)

actor = actor_collection

{'_id': ObjectId('61d937e1f92a9ab995022821'), '배우이름': '김윤석', '흥행지수': 46285, '출연영화': ['모가디슈', '타짜', '당신, 거기 있어줄래요', '화이 : 괴물을 삼킨 아이', '미성년', '1987'], '랭킹': '1', '직업': '배우', '생년월일': '1968-01-21', '성별': '남', '신장/체중': '178cm, 68kg', '학교': '동의대학교 학사', '취미': '여행, 낚시', '소속사': '심엔터테인먼트'}
{'_id': ObjectId('61d937e1f92a9ab995022822'), '배우이름': '조인성', '흥행지수': 40500, '출연영화': ['모가디슈', '클래식'], '랭킹': '2', '직업': '배우', '생년월일': '1981-07-28', '성별': '남', '신장/체중': '186cm, 72kg', '학교': '전남과학대 모델이벤트과 - 동국대 연극영화 (중퇴)', '취미': '모자모으기, 영화보기, 농구', '특기': '농구, 태권도(공인 4단)', '소속사': '싸이더스 HQ', '다른이름': '趙寅成'}
{'_id': ObjectId('61d937e1f92a9ab995022823'), '배우이름': '허준호', '흥행지수': 34683, '출연영화': ['모가디슈', '국가부도의 날'], '랭킹': '3', '원어명': '許俊豪', '직업': '배우', '생년월일': '1964-03-03', '성별': '남', '신장/체중': '180cm, 75kg', '학교': '서울예술대학 연극과', '취미': '만화책 보기', '특기': '야구, 농구', '소속사': '지티비엔터테인먼트'}


### 2.2 Single (단일) 필드 인덱스
- key: ('필드명', direction)
    - direction
        - pymongo.ASCENDING = 1
        - pymongo.DESCENDING = -1
        - pypongo.TEXT = 'text'

In [41]:
actor.create_index('배우이름')

'배우이름_1'

In [40]:
docs = actor.find({'배우이름':'김윤석'})
for doc in docs:
    print(doc)

{'_id': ObjectId('61d937e1f92a9ab995022821'), '배우이름': '김윤석', '흥행지수': 46285, '출연영화': ['모가디슈', '타짜', '당신, 거기 있어줄래요', '화이 : 괴물을 삼킨 아이', '미성년', '1987'], '랭킹': '1', '직업': '배우', '생년월일': '1968-01-21', '성별': '남', '신장/체중': '178cm, 68kg', '학교': '동의대학교 학사', '취미': '여행, 낚시', '소속사': '심엔터테인먼트'}


In [29]:
actor.index_information()

{'_id_': {'v': 2, 'key': [('_id', 1)]},
 '배우이름_1': {'v': 2, 'key': [('배우이름', 1)]}}

In [42]:
actor.create_index('랭킹')
actor.create_index('흥행지수')
actor.index_information()

{'_id_': {'v': 2, 'key': [('_id', 1)]},
 '배우이름_1': {'v': 2, 'key': [('배우이름', 1)]},
 '랭킹_1': {'v': 2, 'key': [('랭킹', 1)]},
 '흥행지수_1': {'v': 2, 'key': [('흥행지수', 1)]}}

In [43]:
# actor.create_index([('직업', pymongo.DESCENDING)])
actor.create_index([('직업', -1)])

'출연영화_text'

#### 부분 문자열 검색을 위해 text index 생성

In [ ]:
# actor.create_index([('출연영화', pymongo.TEXT)])
actor.create_index([('출연영화', 'text')])

In [44]:
actor.index_information()

{'_id_': {'v': 2, 'key': [('_id', 1)]},
 '배우이름_1': {'v': 2, 'key': [('배우이름', 1)]},
 '랭킹_1': {'v': 2, 'key': [('랭킹', 1)]},
 '흥행지수_1': {'v': 2, 'key': [('흥행지수', 1)]},
 '직업_-1': {'v': 2, 'key': [('직업', -1)]},
 '출연영화_text': {'v': 2,
  'key': [('_fts', 'text'), ('_ftsx', 1)],
  'weights': SON([('출연영화', 1)]),
  'default_language': 'english',
  'language_override': 'language',
  'textIndexVersion': 3}}

#### 부분 문자열 검색:  $text - $search
* text 인덱스가 생성된 상태여야 사용할 수 있음

In [45]:
docs = actor.find({'$text': {'$search':'괴물'}})
for doc in docs:
    print(doc)

{'_id': ObjectId('61d937e1f92a9ab9950228a4'), '배우이름': '엄상현', '흥행지수': 1014, '출연영화': ['퍼피 구조대 더 무비', '바다 탐험대 옥토넛 : 육지수호 대작전', '극장판 바다 탐험대 옥토넛: 불의 고리 대폭발', '런닝맨: 풀룰루의 역습', '극장판 바다 탐험대 옥토넛: 대산호초 보호작전', '바다 탐험대 옥토넛 시즌4: 바다 괴물 대소동'], '랭킹': '132', '직업': '성우', '생년월일': '1971-12-29', '성별': '남'}


#### 전체 인덱스 삭제

In [47]:
actor.drop_indexes()

In [48]:
actor.index_information()

{'_id_': {'v': 2, 'key': [('_id', 1)]}}

#### 특정 인덱스 삭제

In [32]:
actor.drop_index([('배우이름', 1)])

In [33]:
actor.index_information()

{'_id_': {'v': 2, 'key': [('_id', 1)]},
 '랭킹_1': {'v': 2, 'key': [('랭킹', 1)]},
 '흥행지수_1': {'v': 2, 'key': [('흥행지수', 1)]},
 '직업_-1': {'v': 2, 'key': [('직업', -1)]},
 '출연영화_text': {'v': 2,
  'key': [('_fts', 'text'), ('_ftsx', 1)],
  'weights': SON([('출연영화', 1)]),
  'default_language': 'english',
  'language_override': 'language',
  'textIndexVersion': 3}}

### 2.3 Compound (복합) 필드 인덱스
* 최대 31개의 필드로 만들 수 있음

In [49]:
actor.create_index([('출연영화', pymongo.TEXT), ('직업', pymongo.TEXT), ('흥행지수', pymongo.DESCENDING)])

'출연영화_text_직업_text_흥행지수_-1'

In [50]:
docs = actor.find({'$text': {'$search':'신과'}})

for doc in docs:
    print(doc)

{'_id': ObjectId('61d937e1f92a9ab995022898'), '배우이름': '김향기', '흥행지수': 1182, '출연영화': ['아이', '신과 함께-죄와 벌', '신과 함께-인과 연', '늑대소년', '증인'], '랭킹': '120', '직업': '배우', '생년월일': '2000-08-09', '성별': '여', '홈페이지': '\nhttps://twitter.com/smell2001\n', '특기': '애교 부리기, 피아노 치기'}


In [51]:
docs = actor.find({'$text': {'$search':'가수'}})

for doc in docs:
    print(doc)

{'_id': ObjectId('61d937e1f92a9ab9950228a6'), '배우이름': '송가인', '흥행지수': 1002, '출연영화': ['송가인 더 드라마'], '랭킹': '134', '직업': '가수', '생년월일': '1986-12-26', '성별': '여', '홈페이지': '\nhttp://instagram.com/songgain_\nhttp://www.facebook.com/eunsim.jo.9\n', '다른이름': '조은심'}
{'_id': ObjectId('61d937e1f92a9ab99502289e'), '배우이름': '한선화', '흥행지수': 1106, '출연영화': ['강릉', '영화의 거리'], '랭킹': '126', '직업': '가수', '생년월일': '1990-10-06', '성별': '여', '홈페이지': '\nhttps://www.instagram.com/shh_daily/\nhttps://twitter.com/seonhwazzz\n', '다른이름': '시크릿'}
{'_id': ObjectId('61d937e1f92a9ab995022896'), '배우이름': '찬열', '흥행지수': 1204, '출연영화': ['더 박스', '장수상회'], '랭킹': '118', '직업': '가수', '생년월일': '1992-11-27', '성별': '남', '홈페이지': '\nhttps://instagram.com/real__pcy/\n', '소속사': 'SM엔터테인먼트', '다른이름': '박찬열; 엑소;EXO'}
{'_id': ObjectId('61d937e1f92a9ab995022887'), '배우이름': '서인국', '흥행지수': 1495, '출연영화': ['파이프라인'], '랭킹': '103', '직업': '가수', '생년월일': '1987-10-23', '성별': '남', '홈페이지': '\nhttps://twitter.com/sigstyle1023\nhttps://www.facebook.com/sigstyle1023\n', '

### 2.4 다양한 문자열 검색

In [53]:
# 정규표현식 ($text operator 는 $search operator 와 함께 사용됨)

result = actor.find({'출연영화' : {'$regex' : '괴물'}})

for record in result:
    print(record)

{'_id': ObjectId('61d937e1f92a9ab995022821'), '배우이름': '김윤석', '흥행지수': 46285, '출연영화': ['모가디슈', '타짜', '당신, 거기 있어줄래요', '화이 : 괴물을 삼킨 아이', '미성년', '1987'], '랭킹': '1', '직업': '배우', '생년월일': '1968-01-21', '성별': '남', '신장/체중': '178cm, 68kg', '학교': '동의대학교 학사', '취미': '여행, 낚시', '소속사': '심엔터테인먼트'}
{'_id': ObjectId('61d937e1f92a9ab99502282a'), '배우이름': '김성균', '흥행지수': 20366, '출연영화': ['싱크홀', '화이 : 괴물을 삼킨 아이', '프리즌'], '랭킹': '10', '직업': '배우', '생년월일': '1980-05-25', '성별': '남', '홈페이지': '\nhttps://twitter.com/ggubun\n'}
{'_id': ObjectId('61d937e1f92a9ab99502283f'), '배우이름': '박용우', '흥행지수': 9265, '출연영화': ['유체이탈자', '화이 : 괴물을 삼킨 아이'], '랭킹': '31', '직업': '배우', '생년월일': '1971-03-16', '성별': '남', '신장/체중': '176cm, 65kg', '학교': '중앙대학교 연극영화과', '취미': '볼링 , 피아노', '특기': '노래, 춤, 태권도', '소속사': '스타파크엔터테인먼트'}
{'_id': ObjectId('61d937e1f92a9ab99502286c'), '배우이름': '조진웅', '흥행지수': 2200, '출연영화': ['1984 최동원', '경관의 피', '독전', '장수상회', '화이 : 괴물을 삼킨 아이', '블랙머니'], '랭킹': '76', '직업': '배우', '생년월일': '1976-03-03', '성별': '남', '홈페이지': '\nhttp://www.face

In [54]:
# 실습을 위해 text_collection 생성

from bs4 import BeautifulSoup
import requests
import pymongo
import re                   

connection = pymongo.MongoClient()
actor_db = connection.cine21
text_collection = actor_db.text_collection

In [55]:
text_collection.insert_many(
    [
        { "name": "Java Hut", "description": "Coffee and cakes", "ranking": 1 },
        { "name": "Burger Buns", "description": "Java hamburgers", "ranking": 2 },
        { "name": "Coffee Shop", "description": "Just coffee", "ranking": 3 },
        { "name": "Clothes Clothes Clothes", "description": "Discount clothing", "ranking": 4 },
        { "name": "Java Shopping", "description": "Indonesian goods", "ranking": 5 }
    ]
)

In [57]:
# + -> 문자가 1개 이상
# * -> 문자가 0개 이상

docs = text_collection.find({'name': {'$regex' : 'Co.+'}})
for doc in docs:
    print(doc)

{'_id': ObjectId('61d9b8a600ca99a8ab45c392'), 'name': 'Coffee Shop', 'description': 'Just coffee', 'ranking': 3}


In [58]:
docs = text_collection.find({'name': {'$regex' : 'Sh.*'}})
for doc in docs:
    print(doc)

{'_id': ObjectId('61d9b8a600ca99a8ab45c392'), 'name': 'Coffee Shop', 'description': 'Just coffee', 'ranking': 3}
{'_id': ObjectId('61d9b8a600ca99a8ab45c394'), 'name': 'Java Shopping', 'description': 'Indonesian goods', 'ranking': 5}


In [59]:
text_collection.create_index([('name', pymongo.TEXT)])

'name_text'

In [60]:
text_collection.index_information()

{'_id_': {'v': 2, 'key': [('_id', 1)]},
 'name_text': {'v': 2,
  'key': [('_fts', 'text'), ('_ftsx', 1)],
  'weights': SON([('name', 1)]),
  'default_language': 'english',
  'language_override': 'language',
  'textIndexVersion': 3}}

In [61]:
docs = text_collection.find({'$text' : {'$search': 'coffee'}})
for doc in docs:
    print(doc)

{'_id': ObjectId('61d9b8a600ca99a8ab45c392'), 'name': 'Coffee Shop', 'description': 'Just coffee', 'ranking': 3}


In [62]:
# 'java', 'coffee', 'shop' 각 각 따로 검색

docs = text_collection.find({'$text' : {'$search': 'java coffee shop'}})
for doc in docs:
    print(doc)

{'_id': ObjectId('61d9b8a600ca99a8ab45c392'), 'name': 'Coffee Shop', 'description': 'Just coffee', 'ranking': 3}
{'_id': ObjectId('61d9b8a600ca99a8ab45c394'), 'name': 'Java Shopping', 'description': 'Indonesian goods', 'ranking': 5}
{'_id': ObjectId('61d9b8a600ca99a8ab45c390'), 'name': 'Java Hut', 'description': 'Coffee and cakes', 'ranking': 1}


In [65]:
# ' \"coffee shop\" ' -> coffee shop 전체를 포함하는 document 출력

docs = text_collection.find({'$text' : {'$search': '\"coffee shop\"'}})
for doc in docs:
    print(doc)

{'_id': ObjectId('61d9b8a600ca99a8ab45c394'), 'name': 'Java Shopping', 'description': 'Indonesian goods', 'ranking': 5}


In [66]:
# '$caseSensitive: True or False'

docs = text_collection.find({'$text' : {'$search': 'coffee shop', '$caseSensitive':True}})
for doc in docs:
    print(doc)

### 실습 

In [86]:
## 중앙대학교를 나온 배우를 흥행지수 순으로 최대 10명 출력하기

docs = actor.find({'학교' : {'$regex': '중앙대학교'}}, {'배우이름':1, '흥행지수':1, '학교':1, '_id':0}).sort('흥행지수', -1).limit(10)

for doc in docs:
    print(doc)

{'배우이름': '강하늘', '흥행지수': 12425, '학교': '중앙대학교 연극학과'}
{'배우이름': '박용우', '흥행지수': 9265, '학교': '중앙대학교 연극영화과'}
{'배우이름': '김강우', '흥행지수': 8815, '학교': '중앙대학교 연극영화학'}
{'배우이름': '진기주', '흥행지수': 1552, '학교': '중앙대학교 컴퓨터공학부'}
{'배우이름': '권율', '흥행지수': 1546, '학교': '중앙대학교 연극학과'}
{'배우이름': '박철민', '흥행지수': 1403, '학교': '중앙대학교 경영학 학사'}
{'배우이름': '이연희', '흥행지수': 1176, '학교': '중앙대학교 연극영화학'}
{'배우이름': '박근형', '흥행지수': 965, '학교': '중앙대학교 연극영화학'}


In [88]:
docs = actor.find({'학교' : {'$regex': '서울'}}, {'배우이름':1, '흥행지수':1, '학교':1, '_id':0}).sort('흥행지수', -1).limit(10)

for doc in docs:
    print(doc)

{'배우이름': '허준호', '흥행지수': 34683, '학교': '서울예술대학 연극과'}
{'배우이름': '황정민', '흥행지수': 24566, '학교': '서울예술대학 연극과 졸업'}
{'배우이름': '류승룡', '흥행지수': 6328, '학교': '서울예술대학 연극'}
{'배우이름': '김희원', '흥행지수': 4986, '학교': '서울예술대학 연극과'}
{'배우이름': '김지호', '흥행지수': 3756, '학교': '서울여자대학교 영문학'}
{'배우이름': '한지민', '흥행지수': 2435, '학교': '서울여자대학교 사회사업학 학사'}
{'배우이름': '장혁', '흥행지수': 2338, '학교': '서울예대 영화 - 단국대 연극영화학'}
{'배우이름': '박희순', '흥행지수': 1763, '학교': '서울예술대학 연극과'}
{'배우이름': '강새봄', '흥행지수': 1021, '학교': '서울대학교 국악과'}
{'배우이름': '정진영', '흥행지수': 918, '학교': '서울대학교 국어국문학'}
